In [ ]:
import pandas as pd
import numpy as np
import json, os
from pandas.io.json import json_normalize
import missingno as msno
import seaborn as sn
from scipy import stats
import datetime as dt
import matplotlib.pyplot as plt
from sklearn import preprocessing
%matplotlib inline

In [11]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
os.chdir('D:/Kaggle/Two Sigma Connect')

In [ ]:
df = pd.read_json('train.json')
print df.head()

In [ ]:
# encoding character lables in target variable

le = preprocessing.LabelEncoder()
le.fit(df.interest_level)
df['target'] = le.transform(df.interest_level)
print df.head()

In [13]:
X = df[['bathrooms', 'bedrooms', 'price']]
y = df['target']

In [14]:
print X.head()
print y.head()

        bathrooms  bedrooms  price
10            1.5         3   3000
10000         1.0         2   5465
100004        1.0         1   2850
100007        1.0         1   3275
100013        1.0         4   3350
10        2
10000     1
100004    0
100007    1
100013    1
Name: target, dtype: int64


In [12]:
est = GradientBoostingClassifier(n_estimators = 100, max_depth = 3)

In [19]:
est.fit(X, y)

GradientBoostingClassifier(init=None, learning_rate=0.1, loss='deviance',
              max_depth=3, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [38]:
# Perform Grid Search 

from sklearn.grid_search import GridSearchCV

In [40]:
param_grid = {'learning_rate':[0.05, 0.02, 0.01],
              'max_depth':[4, 6]}

est = GradientBoostingClassifier(n_estimators = 1000)
gs_cv = GridSearchCV(est, param_grid).fit(X, y)
print gs_cv.best_params_

{'learning_rate': 0.01, 'max_depth': 4}


In [41]:
# apply the tuned parameters to the algorithm and re-run

est_cv = GradientBoostingClassifier(n_estimators = 1000, learning_rate = 0.01, max_depth = 4)
est_cv.fit(X, y)

GradientBoostingClassifier(init=None, learning_rate=0.01, loss='deviance',
              max_depth=4, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=1000,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [35]:
# validate models - Confustion matrix

from sklearn.metrics import confusion_matrix

In [37]:
# predict on to train set and compare with actual classes

pred_train = est.predict(X)
confusion_matrix(pd.DataFrame(pred_train), pd.DataFrame(y))

array([[  515,   181,   195],
       [ 3180, 33933, 10789],
       [  144,   170,   245]])

In [42]:
# with grid search

pred_train_cv = est_cv.predict(X)
confusion_matrix(pred_train_cv, y)

array([[  537,   178,   197],
       [ 3155, 33918, 10733],
       [  147,   188,   299]])

In [33]:
# load test set 

df_test = pd.read_json('test.json')
test_X = df_test[['bedrooms', 'bathrooms', 'price']]
print df_test.head()

        bathrooms  bedrooms                       building_id  \
0             1.0         1  79780be1514f645d7e6be99a3de696c5   
1             1.0         2                                 0   
100           1.0         1  3dbbb69fd52e0d25131aa1cd459c87eb   
1000          1.0         2  783d21d013a7e655bddc4ed0d461cc5e   
100000        2.0         2  6134e7c4dd1a98d9aee36623c9872b49   

                    created  \
0       2016-06-11 05:29:41   
1       2016-06-24 06:36:34   
100     2016-06-03 04:29:40   
1000    2016-06-11 06:17:35   
100000  2016-04-12 05:24:17   

                                              description  \
0       Large with awesome terrace--accessible via bed...   
1       Prime Soho - between Bleecker and Houston - Ne...   
100             New York chic has reached a new level ...   
1000    Step into this fantastic new Construction in t...   
100000  ~Take a stroll in Central Park, enjoy the ente...   

              display_address  \
0              Suffolk

In [43]:
pred = est_cv.predict_proba(test_X)
print pred

[[ 0.03570335  0.72566587  0.23863078]
 [ 0.03530439  0.70401665  0.26067895]
 [ 0.00962076  0.91058945  0.07978979]
 ..., 
 [ 0.00789062  0.98247803  0.00963134]
 [ 0.17894666  0.48178766  0.33926568]
 [ 0.03530439  0.70401665  0.26067895]]


In [44]:
pred_df = pd.DataFrame(pred, columns = ('high', 'medium', 'low'))
print pred_df.head()

       high    medium       low
0  0.035703  0.725666  0.238631
1  0.035304  0.704017  0.260679
2  0.009621  0.910589  0.079790
3  0.022877  0.803753  0.173370
4  0.021396  0.847264  0.131340


In [45]:
pred_df['listing_id'] = df_test.listing_id.values
pred_df.to_csv('submission2_GS_GB.csv', index = False)